The goal here is to load the exgalm5 data for all the 6-bands and restructure them so that we have the median depths for all the filters in one file. Before that, I test whether the saved i-band depth matches that from the OSWG summary excel sheet.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
import time
import copy
import numpy as np
import healpy as hp
import os

%matplotlib inline

In [3]:
fontsize = 18
rcparams = {}
rcparams['figure.figsize'] = (10, 6)
rcparams['axes.labelsize'] = fontsize
rcparams['legend.fontsize'] = fontsize-4
rcparams['axes.titlesize'] = fontsize
rcparams['axes.linewidth'] = 2
rcparams['axes.grid'] = True
for axis in ['x', 'y']:
    rcparams['%stick.labelsize'%axis] = fontsize-2
    rcparams['%stick.direction'%axis] = 'in'
    rcparams['%stick.major.size'%axis] = 5.5
    rcparams['%stick.minor.size'%axis] =  3.5
    rcparams['%stick.major.width'%axis] = 2
    rcparams['%stick.minor.width'%axis] = 1.5
rcparams['xtick.top'] = True
rcparams['ytick.right'] = True
for key in rcparams: mpl.rcParams[key] = rcparams[key]

In [4]:
path = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.6_-0.1cuts/exgalm5_data/summary_data/'
#os.listdir(path)

In [5]:
outdir = 'paper-data/exgalm5-related_v1.6/'
os.makedirs(outdir, exist_ok=True)

##### read in just the i-band data first

In [6]:
yr_tags = ['y1', 'y3', 'y6', 'y10']

new_data = {}
for yr_tag in yr_tags:
    fname = [f for f in os.listdir(path) if f.__contains__('_i-band_') and f.__contains__('_%s_' % yr_tag) and f.endswith('csv')][0]
    print('reading in %s' % fname)
    new_data[yr_tag] = pd.read_csv('%s/%s' % (path, fname))
    print(yr_tag)

reading in median-depth_i-band_eg-pixels_y1_limi24.65_nside256.csv
y1
reading in median-depth_i-band_eg-pixels_y3_limi25.25_nside256.csv
y3
reading in median-depth_i-band_eg-pixels_y6_limi25.619999999999997_nside256.csv
y6
reading in median-depth_i-band_eg-pixels_y10_limi25.9_nside256.csv
y10


##### read in the summary data 

In [7]:
summary_data = pd.read_csv('paper-data/summary_csv_v1.6_-0.1cuts/opsim_v1.6_all-metrics_new.csv')
summary_data

,Unnamed: 0,static_median Y1 $i$-band coadded depth in effective survey area,static_median Y3 $i$-band coadded depth in effective survey area,static_median Y6 $i$-band coadded depth in effective survey area,static_median Y10 $i$-band coadded depth in effective survey area,static_stddev of Y1 $i$-band coadded depth in effective survey area,static_stddev of Y3 $i$-band coadded depth in effective survey area,static_stddev of Y6 $i$-band coadded depth in effective survey area,static_stddev of Y10 $i$-band coadded depth in effective survey area,static_Y1 effective survey area,static_Y3 effective survey area,static_Y6 effective survey area,static_Y10 effective survey area,lss_diagnostic FOM Y1,lss_diagnostic FOM Y10,lss_Y1 Ngal (0.66<z<1.0),lss_Y10 Ngal (0.66<z<1.0)
0,barebones_nexp2_v1.6_10yrs.db,25.20,25.81,26.14,26.40,0.19,0.15,0.14,0.13,15163.15,15326.81,15288.94,15282.23,0.893868,0.975746,444000000,959000000
1,barebones_v1.6_10yrs.db,25.21,25.86,26.19,26.46,0.19,0.15,0.14,0.13,15198.09,15382.68,15355.87,15339.35,0.924743,0.980020,445000000,964000000
2,baseline_nexp1_v1.6_10yrs.db,25.11,25.73,26.07,26.34,0.18,0.13,0.13,0.12,15165.04,15390.34,15353.99,15334.00,0.935198,0.979563,443000000,961000000
3,baseline_nexp2_scaleddown_v1.6_10yrs.db,25.06,25.69,26.03,26.30,0.17,0.13,0.13,0.12,14863.26,15257.10,15177.94,15178.68,0.915822,0.974355,434000000,950000000
4,baseline_nexp2_v1.6_10yrs.db,25.03,25.67,26.01,26.27,0.17,0.13,0.12,0.12,14836.77,15246.82,15188.64,15159.64,0.912427,0.973666,433000000,948000000
5,combo_dust_nexp2_v1.6_10yrs.db,24.97,25.56,25.91,26.18,0.18,0.13,0.13,0.12,15765.87,17556.45,17194.56,17087.49,0.916119,1.030839,459000000,1060000000
6,combo_dust_v1.6_10yrs.db,25.03,25.63,25.98,26.24,0.19,0.14,0.14,0.13,17161.14,18024.20,17771.62,17693.36,0.957700,1.049415,500000000,1100000000
7,ddf_heavy_nexp2_v1.6_10yrs.db,24.98,25.56,25.89,26.16,0.14,0.10,0.10,0.09,14262.06,15029.02,14827.70,14765.80,0.898389,0.962697,415000000,919000000
8,ddf_heavy_v1.6_10yrs.db,25.06,25.64,25.97,26.23,0.16,0.12,0.11,0.10,14944.15,15235.75,15129.58,15089.35,0.922243,0.973398,436000000,942000000
9,dm_heavy_nexp2_v1.6_10yrs.db,24.99,25.65,26.00,26.27,0.16,0.13,0.12,0.11,14330.89,15177.68,15150.25,15150.09,0.895523,0.972498,417000000,947000000


In [8]:
#summary_data = summary_data.drop(index=0, axis=0)

#### compare the overlapping data (area, i-band depth, depth stddev) between the two files

look to see if all the dbs in the new data are presented in the summary data

In [9]:
for yr_tag in yr_tags:
    print('checking %s' % yr_tag)
    mapper = {'i-band depth: median': 'static_median Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1], 
          'Area (deg2)': 'static_Y%s effective survey area'  % yr_tag.split('y')[-1],
          'i-band depth: std': 'static_stddev of Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1]
         }
    
    inds = []
    for i, db in enumerate( new_data[yr_tag]['dbname'] ):
        ind = np.where(summary_data['Unnamed: 0'] == '%s.db' % db)[0]
        inds.append(ind[0])
        for stat_key in mapper:
            #print('## "%s" vs. "%s"' % (stat_key, mapper[stat_key]))
            val1 = new_data[yr_tag][stat_key].values[i]
            val2 = float(summary_data[mapper[stat_key]].values[ind][0])
            if val1 != val2:
                print('things dont match for %s: %s: %s vs. %s' % (db, stat_key, val1, val2))

checking y1
checking y3
checking y6
checking y10


look to see if all the dbs in the all-metrics data are presented in the new data

In [10]:
for yr_tag in yr_tags:
    print('\nchecking %s' % yr_tag)
    mapper = {'i-band depth: median': 'static_median Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1], 
          'Area (deg2)': 'static_Y%s effective survey area'  % yr_tag.split('y')[-1],
          'i-band depth: std': 'static_stddev of Y%s $i$-band coadded depth in effective survey area' % yr_tag.split('y')[-1]
         }
    
    inds = []
    for i, db in enumerate( summary_data['Unnamed: 0'] ):
        ind = np.where(new_data[yr_tag]['dbname'] == db.split('.db')[0])[0]
        
        if len(ind)  == 0:
            print('%s not in the new data' % db)
        
        inds.append(ind)
        for stat_key in mapper:
            #print('## "%s" vs. "%s"' % (stat_key, mapper[stat_key]))
            val1 = new_data[yr_tag][stat_key].values[ind]
            val2 = float(summary_data[mapper[stat_key]].values[i])
            if val1 != val2:
                print('things dont match for %s: %s: %s vs. %s' % (db, stat_key, val1, val2))


checking y1

checking y3

checking y6

checking y10


confirm that all the dbs are present for all the years.

one more check: number of dbs for each year:

In [11]:
for yr_tag in yr_tags:
    print(yr_tag, len(new_data[yr_tag]['dbname']))

y1 30
y3 30
y6 30
y10 30


okay things match -- good. can proceed to saving things now.

#### assemble the data to save

need to make sure that the dbs for all the bands are added in the same order for all years

In [12]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']

In [13]:
depth_med_all_yrs = {}
depth_stddev_all_yrs = {}

for yr_tag in ['y10', 'y3', 'y6', 'y1']:
    depth_med_this_yr = pd.DataFrame()
    depth_stddev_this_yr = pd.DataFrame()
    
    for i, band in enumerate( bands ):
        fname = [f for f in os.listdir(path) if f.__contains__('_%s-band_' % band) \
                                                 and f.__contains__('_%s_' % yr_tag)
                ][0]
        print('reading in %s' % fname)
        data_ = pd.read_csv('%s/%s' % (path, fname))

        band_key_median = '%s-band median coadded dust-corrected depth in eg-footprint' % band
        band_key_stddev = '%s-band coadded dust-corrected depth stddev in eg-footprint' % band
        if i == 0:
            # add to the all-data dictionary
            # median depth
            depth_med_this_yr['dbname'] = data_['dbname'].values
            depth_med_this_yr['yr'] = [yr_tag] * len( data_['dbname'].values )
            # depth stddev
            depth_stddev_this_yr['dbname'] = data_['dbname'].values
            depth_stddev_this_yr['yr'] = [yr_tag] * len( data_['dbname'].values )
        else:
            # check if the order is okay
            
            if (depth_med_this_yr['dbname'].values != data_['dbname'].values).any():
                print('\n## dbname order is different for %s-band vs. previous bands for %s.' % (band, yr_tag) )
                
                print('reordering ... ')
                
                # reorder things
                data_ = depth_med_this_yr.merge(data_, on='dbname', suffixes=['_prev', '_%s' % yr_tag])
                
                
                # check to ensure the db order is the same
                print('checking the order again .. should not print anything if the order is now fine.\n')
                if  (depth_med_this_yr['dbname'].values != data_['dbname'].values).any():
                    raise ValueError('somethings wrong with the db order')
                
        depth_med_this_yr[band_key_median] = data_['%s-band depth: median' % band].values
        depth_stddev_this_yr[band_key_stddev] = data_['%s-band depth: std' % band].values
            
    depth_med_all_yrs[yr_tag] = depth_med_this_yr
    depth_stddev_all_yrs[yr_tag] = depth_stddev_this_yr

reading in median-depth_u-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_g-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_r-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_i-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_z-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_y-band_eg-pixels_y10_limi25.9_nside256.csv
reading in median-depth_u-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_g-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_r-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_i-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_z-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_y-band_eg-pixels_y3_limi25.25_nside256.csv
reading in median-depth_u-band_eg-pixels_y6_limi25.619999999999997_nside256.csv
reading in median-depth_g-band_eg-pixels_y6_limi25.619999999999997_nside256.csv
reading in median-depth_r-band_eg-pi

order the dbs for y1, y3, y6 as is for y10

In [14]:
#### assemble data to save first in the order of each years pd df

In [15]:
y10_order = pd.DataFrame({'dbname': depth_med_all_yrs['y10']['dbname']})

In [16]:
# median depth data - move away from the nested dictionary
depth_data = pd.DataFrame()
for i, yr_tag in enumerate( depth_med_all_yrs ):
    depth_med_all_yrs[yr_tag] = y10_order.merge(depth_med_all_yrs[yr_tag], on='dbname')
    
    if i == 0:
        depth_data = depth_med_all_yrs[yr_tag]
    else:
        # stack them up
        depth_data = depth_data.append(depth_med_all_yrs[yr_tag])
        
depth_data.reset_index(inplace=True, drop=True)

depth_data

,dbname,yr,u-band median coadded dust-corrected depth in eg-footprint,g-band median coadded dust-corrected depth in eg-footprint,r-band median coadded dust-corrected depth in eg-footprint,i-band median coadded dust-corrected depth in eg-footprint,z-band median coadded dust-corrected depth in eg-footprint,y-band median coadded dust-corrected depth in eg-footprint
0,mw_heavy_nexp2_v1.6_10yrs,y10,25.41,26.71,26.78,26.26,25.57,24.80
1,rolling_fpo_2nslice1.0_v1.6_10yrs,y10,25.65,26.83,26.85,26.32,25.62,24.86
2,dm_heavy_nexp2_v1.6_10yrs,y10,25.29,26.61,26.71,26.27,25.59,24.77
3,rolling_fpo_2nslice0.9_v1.6_10yrs,y10,25.65,26.83,26.85,26.32,25.61,24.86
4,ddf_heavy_nexp2_v1.6_10yrs,y10,25.35,26.64,26.71,26.16,25.50,24.75
...,...,...,...,...,...,...,...,...
115,rolling_fpo_6nslice0.9_v1.6_10yrs,y1,24.39,25.54,25.66,25.14,24.43,23.85
116,even_filters_g_v1.6_10yrs,y1,24.39,25.51,25.53,25.12,24.37,23.78
117,rolling_fpo_3nslice1.0_v1.6_10yrs,y1,24.40,25.57,25.65,25.13,24.41,23.84
118,mw_heavy_v1.6_10yrs,y1,24.39,25.56,25.63,25.11,24.37,23.83


In [17]:
# depth stddev data
stddev_data = pd.DataFrame()
for i, yr_tag in enumerate( depth_stddev_all_yrs ):
    depth_stddev_all_yrs[yr_tag] = y10_order.merge(depth_stddev_all_yrs[yr_tag], on='dbname')
    
    if i == 0:
        stddev_data = depth_stddev_all_yrs[yr_tag]
    else:
        # stack them up
        stddev_data = stddev_data.append(depth_stddev_all_yrs[yr_tag])
        
stddev_data.reset_index(inplace=True, drop=True)

stddev_data

,dbname,yr,u-band coadded dust-corrected depth stddev in eg-footprint,g-band coadded dust-corrected depth stddev in eg-footprint,r-band coadded dust-corrected depth stddev in eg-footprint,i-band coadded dust-corrected depth stddev in eg-footprint,z-band coadded dust-corrected depth stddev in eg-footprint,y-band coadded dust-corrected depth stddev in eg-footprint
0,mw_heavy_nexp2_v1.6_10yrs,y10,0.23,0.19,0.14,0.11,0.10,0.11
1,rolling_fpo_2nslice1.0_v1.6_10yrs,y10,0.23,0.19,0.15,0.12,0.11,0.12
2,dm_heavy_nexp2_v1.6_10yrs,y10,0.25,0.20,0.15,0.11,0.11,0.10
3,rolling_fpo_2nslice0.9_v1.6_10yrs,y10,0.24,0.20,0.15,0.12,0.11,0.12
4,ddf_heavy_nexp2_v1.6_10yrs,y10,0.20,0.16,0.12,0.09,0.09,0.09
...,...,...,...,...,...,...,...,...
115,rolling_fpo_6nslice0.9_v1.6_10yrs,y1,0.34,0.30,0.19,0.19,0.18,0.17
116,even_filters_g_v1.6_10yrs,y1,0.33,0.30,0.21,0.18,0.18,0.17
117,rolling_fpo_3nslice1.0_v1.6_10yrs,y1,0.35,0.31,0.20,0.18,0.18,0.17
118,mw_heavy_v1.6_10yrs,y1,0.34,0.32,0.19,0.18,0.19,0.17


#### plot things out to see


In [18]:
baseline_key = 'baseline_nexp1_v1.6_10yrs'

In [19]:
ndbs = len(np.where(depth_data['yr'].values == 'y10')[0])
print(ndbs)

for yr_tag in yr_tags:
    baseline_ind = np.where( (depth_data['yr'].values[:] == yr_tag) & (depth_data['dbname'].values[:] == baseline_key) )[0]
    plt.clf()
    for band in bands:
        ind = np.where(depth_data['yr'].values == yr_tag)[0]
        band_key = '%s-band median coadded dust-corrected depth in eg-footprint' % band
        p = plt.plot(depth_data[band_key].values[ind], range(ndbs), '.-', label='%s-band' % band, )
        plt.plot( [ depth_data[band_key].values[baseline_ind] ] * ndbs, range(ndbs), '--', color=p[0].get_color(), )

    plt.plot( [depth_data[band_key].values[baseline_ind]], [0], 'k--', label='respective median depth for %s' % baseline_key)

    ax = plt.gca()
    ax.set_yticks( range(ndbs) )
    ax.set_yticklabels( y10_order['dbname'].values )

    plt.xlabel('median depth (coadded; dust extinguished)')
    plt.legend(bbox_to_anchor=(1,1))
    plt.title('%s median depth in eg-footprint' % yr_tag)
    plt.gcf().set_size_inches(10, 30 )
    fname = 'plot_median-depths_%s.png' % yr_tag
    plt.savefig('%s/%s' % (outdir, fname), bbox_inches='tight', format='png')
    print('## saved %s' % fname)
    plt.close() #plt.show()

30
## saved plot_median-depths_y1.png
## saved plot_median-depths_y3.png
## saved plot_median-depths_y6.png
## saved plot_median-depths_y10.png


In [20]:
ndbs = len(np.where(stddev_data['yr'].values == 'y10')[0])

for yr_tag in yr_tags:
    baseline_ind = np.where( (stddev_data['yr'].values[:] == yr_tag) & (stddev_data['dbname'].values[:] == baseline_key) )[0]
    
    plt.clf()
    for band in bands:
        ind = np.where(stddev_data['yr'].values == yr_tag)[0]
        band_key = '%s-band coadded dust-corrected depth stddev in eg-footprint' % band
        p = plt.plot(stddev_data[band_key].values[ind], range(ndbs), '.-', label='%s-band' % band, )
        plt.plot( [ stddev_data[band_key].values[baseline_ind] ] * ndbs, range(ndbs), '--', color=p[0].get_color(), )

    plt.plot( [stddev_data[band_key].values[baseline_ind]], [0], 'k--', label='respective depth stddev for %s' % baseline_key)

    ax = plt.gca()
    ax.set_yticks( range(ndbs) )
    ax.set_yticklabels( y10_order['dbname'].values )

    plt.xlabel('depth stddev (coadded; dust extinguished)')
    plt.legend(bbox_to_anchor=(1,1))
    plt.title('%s depth stddev in eg-footprint' % yr_tag)
    plt.gcf().set_size_inches(10, 30 )
    fname = 'plot_depth-stddev_%s.png' % yr_tag
    plt.savefig('%s/%s' % (outdir, fname), bbox_inches='tight', format='png')
    print('## saved %s' % fname)
    plt.close() #show()

## saved plot_depth-stddev_y1.png
## saved plot_depth-stddev_y3.png
## saved plot_depth-stddev_y6.png
## saved plot_depth-stddev_y10.png


### save the data

#### for melissa g.

In [21]:
yrs_tag = 'y10-6-3-1'
fname = 'exgalm5-data_median_%sbands_%s.csv' % (len(bands), yrs_tag)
depth_data.to_csv('%s/%s' % (outdir, fname), index=False)

In [22]:
yrs_tag = 'y10-6-3-1'
fname = 'exgalm5-data-stddev_%sbands_%s.csv' % (len(bands), yrs_tag)
stddev_data.to_csv('%s/%s' % (outdir, fname), index=False)

#### for oswg sheet

In [23]:
# if db-order is defined already; one df for everything
for i, yr in enumerate( [1, 3, 6, 10] ):
    depth_data_for_this_yr = depth_data.loc[depth_data['yr'] == 'y%s' % yr]
    stddev_data_for_this_yr = stddev_data.loc[stddev_data['yr'] == 'y%s' % yr]
    
    if i == 0:
        df_in_order = pd.DataFrame({'dbname': summary_data['Unnamed: 0'].values})
        # find the indices to map the given order with the order in the df
        # find the indices to get the order
        ind = []
        for db in df_in_order['dbname']:
            #print(db)
            ind_temp = np.where( depth_data_for_this_yr['dbname'].values == db.split('.db')[0] )[0]

            if len(ind_temp) == 0:
                print('%s not found in data dict' % db)
                #ind_temp = np.array([ 0 ])

            if ind_temp in ind:
                print('%s ind already in. %s' % (ind_temp, db))
            ind +=  list( ind_temp )

        ind = np.array(ind)
        print(ind)
    
    # okay assmeble the new dfs
    for colname in stddev_data_for_this_yr:
        if colname != 'dbname' and colname != 'yr':
             df_in_order['Y%s_%s' % (yr, colname)] = stddev_data_for_this_yr[colname].values[ind]

    for colname in depth_data_for_this_yr:
        if colname != 'dbname' and colname != 'yr':
             df_in_order['Y%s_%s' % (yr, colname)] = depth_data_for_this_yr[colname].values[ind]
                
        
                
fname = 'exgalm5-data_%sbands_y1-3-6-10.csv' % (len(bands))
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)
df_in_order

[29 23 18 24 12 22  9  4 19  2 11 21 13 26 16  0 28  5 17 14  3  1 20 15
 27  7 25 10  8  6]


,dbname,Y1_u-band coadded dust-corrected depth stddev in eg-footprint,Y1_g-band coadded dust-corrected depth stddev in eg-footprint,Y1_r-band coadded dust-corrected depth stddev in eg-footprint,Y1_i-band coadded dust-corrected depth stddev in eg-footprint,Y1_z-band coadded dust-corrected depth stddev in eg-footprint,Y1_y-band coadded dust-corrected depth stddev in eg-footprint,Y1_u-band median coadded dust-corrected depth in eg-footprint,Y1_g-band median coadded dust-corrected depth in eg-footprint,Y1_r-band median coadded dust-corrected depth in eg-footprint,...,Y10_r-band coadded dust-corrected depth stddev in eg-footprint,Y10_i-band coadded dust-corrected depth stddev in eg-footprint,Y10_z-band coadded dust-corrected depth stddev in eg-footprint,Y10_y-band coadded dust-corrected depth stddev in eg-footprint,Y10_u-band median coadded dust-corrected depth in eg-footprint,Y10_g-band median coadded dust-corrected depth in eg-footprint,Y10_r-band median coadded dust-corrected depth in eg-footprint,Y10_i-band median coadded dust-corrected depth in eg-footprint,Y10_z-band median coadded dust-corrected depth in eg-footprint,Y10_y-band median coadded dust-corrected depth in eg-footprint
0,barebones_nexp2_v1.6_10yrs.db,0.38,0.31,0.20,0.19,0.19,0.17,24.22,25.57,25.62,...,0.15,0.13,0.12,0.12,25.52,26.80,26.85,26.40,25.64,24.91
1,barebones_v1.6_10yrs.db,0.38,0.33,0.23,0.19,0.20,0.15,24.49,25.69,25.71,...,0.16,0.13,0.13,0.12,25.76,26.94,26.93,26.46,25.69,24.98
2,baseline_nexp1_v1.6_10yrs.db,0.32,0.31,0.19,0.18,0.18,0.18,24.41,25.59,25.63,...,0.15,0.12,0.11,0.13,25.66,26.85,26.87,26.34,25.64,24.86
3,baseline_nexp2_scaleddown_v1.6_10yrs.db,0.32,0.28,0.18,0.17,0.18,0.16,24.19,25.47,25.55,...,0.15,0.12,0.11,0.10,25.44,26.75,26.81,26.30,25.60,24.82
4,baseline_nexp2_v1.6_10yrs.db,0.33,0.32,0.18,0.17,0.18,0.17,24.17,25.48,25.53,...,0.14,0.12,0.11,0.11,25.42,26.73,26.79,26.27,25.58,24.82
5,combo_dust_nexp2_v1.6_10yrs.db,0.34,0.31,0.20,0.18,0.20,0.19,24.09,25.34,25.46,...,0.15,0.12,0.12,0.12,25.33,26.64,26.70,26.18,25.49,24.74
6,combo_dust_v1.6_10yrs.db,0.35,0.32,0.21,0.19,0.20,0.18,24.31,25.45,25.53,...,0.16,0.13,0.13,0.13,25.55,26.75,26.77,26.24,25.54,24.79
7,ddf_heavy_nexp2_v1.6_10yrs.db,0.30,0.28,0.16,0.14,0.16,0.16,24.19,25.41,25.49,...,0.12,0.09,0.09,0.09,25.35,26.64,26.71,26.16,25.50,24.75
8,ddf_heavy_v1.6_10yrs.db,0.35,0.31,0.18,0.16,0.17,0.17,24.34,25.51,25.58,...,0.13,0.10,0.10,0.10,25.58,26.76,26.79,26.23,25.56,24.80
9,dm_heavy_nexp2_v1.6_10yrs.db,0.39,0.42,0.18,0.16,0.19,0.15,24.14,25.46,25.46,...,0.15,0.11,0.11,0.10,25.29,26.61,26.71,26.27,25.59,24.77


okay good -- the dbs are in the same order as before and the values for them match between the older file and the newer one

#### extra things

##### check to confirm the new data saved for y10 matches that saved earlier

#### lets plot out the stddev in depth

#### just for a sanity check: lets plot out the area values in each file; they should match.

okay good. the area values in the files are the same as they should be.